In [1]:
from collections import Counter
import os,re,string,json
from tqdm import tqdm
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import nltk
from matplotlib.dates import date2num
import matplotlib.dates as mdates

from functions import *

plotting.style_()

tf = pd.read_csv(base_path + '/resources/keywords-expertise-translation.csv')
tf_science = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}
tf_science = {k:{x:i for x,i in v.items() if x not in ["policy","program","measures"]} for k,v in tf_science.items()}

In [16]:
def load_clean(iso='gb',start_month='2020-01',end_month='2020-12',text_version='lemmatized'):
    df = data_loader.load_month(iso,start_month,end_month)
    df['ortext'] = df['text']
    df['text'] = [utils.preprocess(str(x)) for x in df[text_version]]
    return df 

def add_metadata(df,iso='gb',house=True,date_format='week'):
    if house == True:
        if iso != 'gb':
            df['house'] = ['upper' if 'upper' in x else 'lower' for x in df['term']]
        if iso == 'gb':
            df['house'] = ['upper' if 'lords' in x else 'lower' for x in df['id']]
    if date_format == 'week':
        df['week'] = [df['from'][c][:4] + '-' + str(datetime.strptime(x, '%Y-%m-%d').isocalendar()[1]) for c,x in enumerate(df['from'])]
        df['week'] = [x.replace('-','-0') if len(x) == 6 else x for x in df['week']]
    if date_format == 'month':
        df['month'] = [x[:7] for x in df['from']]
    if date_format == 'day':
        df['day'] = df['from']
    return df

In [38]:
df = load_clean('it','2020-03','2020-05')
df = add_metadata(df,iso='it')

In [39]:

term1 = 'esperto'
term2 = 'virus'
for i,row in df[(df.text.str.contains(term1)) & (df.text.str.contains(term2))].iterrows():
    print('------')
    print(row['speaker_party'],row['speaker_name'],row['from'])
    print('=================')
    sentence = row['ortext'].split(' ')
    try:
        for x in [" ".join(sentence[c-10:c+10]) for c,i in enumerate(sentence) if 'expert' in i]:
            print(x)
    except:
        continue

------
M5S.2 Pirro, Elisa 2020-03-04
------
IV-PSI Sbrollini, Daniela 2020-03-04
------
PD Boldrini, Paola 2020-03-04
------
M5S.2 Castellone, Maria Domenica 2020-03-04
------
- Gualtieri, Roberto 2020-03-11
------
L-SP-PSd'Az Fregolent, Sonia 2020-04-01
------
- Speranza, Roberto 2020-04-01
------
PD Boldrini, Paola 2020-04-08
------
IV-PSI Comincini, Eugenio 2020-04-08
------
Misto Fattori, Elena 2020-04-16
------
- Conte, Giuseppe 2020-04-21
------
FIBP-UDC Giammanco, Gabriella 2020-04-29
------
M5S.2 Evangelista, Elvira Lucia 2020-04-29
------
- Conte, Giuseppe 2020-04-30
------
Aut (SVP-PATT, UV) Unterberger, Julia 2020-04-30
------
IV-PSI Renzi, Matteo 2020-04-30
------
FdI La Russa, Ignazio 2020-04-30
------
L-SP-PSd'Az Salvini, Matteo 2020-04-30


['that the Care Quality Commission will continue to provide independent expert regulation of health and care providers. It has already']